In [126]:
######################### Imports #########################
import math
import numpy as np

try:
    import utility as ut
    from units import unit_registry as u
except ModuleNotFoundError:
    from AguaClara_design import utility as ut
    from AguaClara_design.units import unit_registry as u


In [127]:
#This pump rotor radius was set based on minimizing the error between predicted and measured values.
# This is for the 6 roller pump heads.
RADIUS_PUMP = 1.62*u.cm

#This corrects for the fact that larger diameter tubing has more loss due to the space smashed by the rollers
#I'm actually not quite sure what this mechanism is..
#This was set based on minimizing the error between predicted and measured values.
K_NONLINEAR = 13

REV = 6.238

In [128]:
TUBE_CODE = ["orange=black","orange-red","orange-blue","orange-green","green-yellow","orange-yellow","white-yellow","orange-white","black-black","orange-orange","white-black","white-white","white-red","red-red","red-grey","grey-grey","yellow-yellow","yellow-blue","blue-blue","blue-green","green-green","purple-purple","purple-black","purple-orange","purple-orange","purple-white","black-white"]

ID_TUBE_ARRAY = [0.13,0.19,0.25,0.38,0.44,0.51,0.57,0.64,0.76,0.89,0.95,1.02,1.09,1.14,1.22,1.30,1.42,1.52,1.65,1.75,1.85,2.06,2.29,2.54,2.62,2.79,3.17]*u.mm

FLOW_TUBE_PER_REV = [0.001383333,0.002833333,0.004833333,0.010833333,0.014333333,0.02,0.023333333,0.03,0.041666667,0.056666667,0.063333333,0.073333333,0.083333333,0.09,0.101666667,0.115,0.133333333,0.151666667,0.166666667,0.183333333,0.216666667,0.25,0.283333333,0.333333333,0.366666667,0.401,0.433333333]*u.mL/REV

TUBING_DATA = [TUBE_CODE, ID_TUBE_ARRAY, FLOW_TUBE_PER_REV]

In [129]:
#The function below returns the flow per revolution given the ID of the 3-stop tubing.
#This function returns values that are very close to the table produced by ismatec.

def flow_6roller_tube_per_rev_calc(ID_tube):
    """returns the flow per revolution given the ID of the 3-stop tubing.
    This function returns values that are very close to the table produced by ismatec."""
    
    return (((RADIUS_PUMP*math.pi*2)-(K_NONLINEAR*ID_tube))/(REV)*((ID_tube)**2*math.pi/4))

#In the lab we currently have orange-yellow, yellow-blue, and purple-white tubing

# Determining Q.floc:

In [130]:
FLOW_PLANT = 7*u.mL/u.s

### Mixing

Clay

In [131]:
#Turbidity (NTU) of the expected influen
NTU = 1.7*u.mg/u.L

#The concentration of clay in the mixed water in the flocculator.
CONC_CLAY = 100*NTU

#This is for a 100 rpm pump and is designed to get the pump operating 
#at an rpm where it has good speed control
#and it has the ability to be adjusted up or down as needed. 
#Some of our pumps have a minimum speed of about 2.3 rpm.
MIN_RPM = 3*REV/u.min
MAX_RPM = 95*REV/u.min

CLAY_TUBING_COLORS = "yellow-blue" #yellow blue tubing

ID_CLAY_TUBING = ID_TUBE_ARRAY[TUBE_CODE.index(CLAY_TUBING_COLORS)]

CONC_CLAY_STOCK_MAX = CONC_CLAY*FLOW_PLANT/(flow_6roller_tube_per_rev_calc(ID_CLAY_TUBING)*MIN_RPM)

CONC_CLAY_STOCK_MAX = ((CONC_CLAY_STOCK_MAX).to(u.g/u.L))

CLAY_STOCK_MAX_EMPIRICAL = 300*u.g/u.L

#Clay stock concentrations of up to 300 g/L have been tested and are able to be pumped

CONC_CLAY_STOCK_MIN = CONC_CLAY*FLOW_PLANT/(flow_6roller_tube_per_rev_calc(ID_CLAY_TUBING)*MAX_RPM)

CONC_CLAY_STOCK_MIN = ((CONC_CLAY_STOCK_MIN).to(u.g/u.L))

#The concentration of clay in the stock bottle.
#You can change this to make sure the designed velocity for the 
#clay pump is within the range of velocity of the pump.
clay_param = 10**((math.floor(math.log10(min(CONC_CLAY_STOCK_MAX,CLAY_STOCK_MAX_EMPIRICAL).magnitude)))-1)
clay_param2 = min(CONC_CLAY_STOCK_MAX,CLAY_STOCK_MAX_EMPIRICAL)/clay_param
CONC_CLAY_STOCK = math.floor(clay_param2.magnitude)*(clay_param)
CONC_CLAY_STOCK = CONC_CLAY_STOCK*(u.g/u.L)

#The volume of container which holds clay suspension. 
VOLUME_CLAY_STOCK = 1*u.L

#The mass of clay that we need to add into the stock container
MASS_STOCK_CLAY = CONC_CLAY_STOCK*VOLUME_CLAY_STOCK

#Flow rate of clay suspension that needed
FLOW_CLAY_STOCK = CONC_CLAY*FLOW_PLANT/CONC_CLAY_STOCK
FLOW_CLAY_STOCK= (FLOW_CLAY_STOCK).to(u.mL/u.s)

#The time at which the clay suspension in the container would run out.
TIME_CLAY_STOCK = VOLUME_CLAY_STOCK/FLOW_CLAY_STOCK
TIME_CLAY_STOCK = TIME_CLAY_STOCK.to(u.hr)



PACl

In [ ]:
#Concentration of Aluminum in the laboratory PACl stock 
#(Confirm that this matches the stock concentration you are using.)
CONC_SUPER_STOCK_PACl = 69.4*u.g/u.L


